## Setup

In [22]:
with open('../openai_key.txt', 'r') as f:
    OPENAI_API_KEY = f.read().strip()
with open('../claude_key.txt', 'r') as f:
    ANTHROPIC_API_KEY = f.read().strip()
with open('../gemini_key.txt', 'r') as f:
    GEMINI_API_KEY = f.read().strip()

COIN_PROMPT = 'Toss an unbiased coin. Output nothing except "Heads" or "Tails". They should each be equally likely. Do not use any tools, just output the result.'


## OpenAI

In [52]:
import openai
import os
import numpy as np

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

openai_client = openai.OpenAI()

completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": COIN_PROMPT
        }
    ],
    logprobs=True,
    top_logprobs=5,
)

for token in completion.choices[0].logprobs.content[0].top_logprobs:
    print(f'{token.token}: {np.exp(token.logprob):.2f}')

Heads: 0.68
T: 0.32
Sure: 0.00
To: 0.00
Certainly: 0.00


## Gemini

In [44]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-1.5-pro")

from tqdm import tqdm

def count_coin_tosses(sample_fn, num_attempts=10):
    head_count = 0
    tail_count = 0
    other_count = 0
    for _ in tqdm(range(num_attempts)):
        text = sample_fn()
        if text.startswith('Heads'):
            head_count += 1
        elif text.startswith('Tails'):
            tail_count += 1
        else:
            other_count += 1
            print(fr'Other: {text}')
    print(f'Heads: {head_count / num_attempts:.2f}, Tails: {tail_count / num_attempts:.2f}, Other: {other_count / num_attempts:.2f}')


count_coin_tosses(lambda: gemini_model.generate_content(COIN_PROMPT).text, num_attempts=20)


100%|██████████| 20/20 [00:13<00:00,  1.51it/s]

Heads: 1.00, Tails: 0.00, Other: 0.00


## Claude

In [43]:
import anthropic


anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def sample_fn():
    message = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": COIN_PROMPT
                    }
                ]
            }
        ]
    )
    return message.content[0].text

count_coin_tosses(sample_fn, num_attempts=20)


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:09<00:00,  2.08it/s]

Heads: 0.65, Tails: 0.35, Other: 0.00
